Impoer librRIES

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

Launch Spark Context

In [2]:
import findspark
findspark.init()
print findspark.find()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

/usr/local/opt/apache-spark/libexec


Check child execution processes

In [3]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x8

In [4]:
from pyspark.sql import SQLContext, Row
sqlsc=SQLContext(sc)

In [5]:
#tweetlines = sc.textFile('./tweets.csv')
#tweetfields = tweetlines.map(lambda l: split(","))
#alltweets = tweetfields.map(lambda t: Row(bounding_box=t[1], city=t[2], country=t[3], text=t[4], tid=t[5], timestamp=t[6], uid=t[7]))
#schemaAlltweets = sqlsc.createDataFrame(alltweets)
#schemaAlltweets.registerTempTable("alltweetsdf")


In [6]:
pandas_df = pd.read_csv('./tweets.csv')
#pandas_df.head(5)
alltweetsdf = sqlsc.createDataFrame(pandas_df)
alltweetsdf.show(5)


+----------+--------------------+----------------+-------------+--------------------+--------------------+-----------------+-------------+
|Unnamed: 0|        bounding_box|            city|      country|                text|                 tid|        timestamp|          uid|
+----------+--------------------+----------------+-------------+--------------------+--------------------+-----------------+-------------+
|         0|{u'type': u'Polyg...|Huntington Beach|United States| In need of back rub|6.693751389036666...| 1.44842644416E12|1.611125173E9|
|         1|{u'type': u'Polyg...|    Wheelersburg|United States|And I'm always ti...|6.693751389583073...|1.448426444173E12|3.165522051E9|
|         2|{u'type': u'Polyg...|        Cascavel|       Brasil|    @DudaDalmas eita|6.693751385556336...|1.448426444077E12| 1.71176445E8|
|         3|{u'type': u'Polyg...|        Rubidoux|United States|Lugo: I want to o...|6.693751377166377E17|1.448426443877E12|2.260612922E9|
|         4|{u'type': u'Pol

In [7]:
alltweetsdf.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- bounding_box: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- text: string (nullable = true)
 |-- tid: double (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- uid: double (nullable = true)



In [8]:
#ustweetsdf = alltweetsdf[alltweetsdf['country'] == "United States"]
ustweetsdf = alltweetsdf.filter(alltweetsdf['country'] == "United States")

<class 'pandas.core.frame.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [19]:
print type(pandas_df)
print pandas_df.count()
print type(alltweetsdf)
print type(ustweetsdf)
print "Count of All tweets =", alltweetsdf.count()
print "Count of US tweets:", ustweetsdf.count()
ustweetsdf.show(5)

<class 'pandas.core.frame.DataFrame'>
Unnamed: 0      35473
bounding_box    35460
city            35460
country         35460
text            35461
tid             35456
timestamp       35456
uid             35456
dtype: int64
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
Count of All tweets = 35473
Count of US tweets: 21808
+----------+--------------------+----------------+-------------+--------------------+--------------------+-----------------+-------------+
|Unnamed: 0|        bounding_box|            city|      country|                text|                 tid|        timestamp|          uid|
+----------+--------------------+----------------+-------------+--------------------+--------------------+-----------------+-------------+
|         0|{u'type': u'Polyg...|Huntington Beach|United States| In need of back rub|6.693751389036666...| 1.44842644416E12|1.611125173E9|
|         1|{u'type': u'Polyg...|    Wheelersburg|United States|And I'm always 

####function to process tweet texts

In [10]:
def process_tweets(text):
    ewe

In [11]:
tweets_processed = []
tweet_texts = ustweetsdf.map(lambda r: r['text'])
for tweet_text in tweet_texts.collect():
    processed_tweet = process_tweets(tweet_text)
    tweets_processed.append(processed_tweet)
    
## OR    
tweets_processed = ustweetsdf.map(lambda r: process_tweets(r.text))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 16, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-11-2c17b41687f2>", line 2, in <lambda>
TypeError: tuple indices must be integers, not str

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.sc